In [1]:
import numpy as np
import os
import pandas as pd
import mne

from mne.preprocessing import ICA

In [2]:
#definisco il percorso iniziale comune 
home_path = os.path.abspath(os.getcwd())

#definisco una lista di nomi per salvare i file
savename=[]
for j in range(50):
    savename.append('paziente'+str(j+1)+'.fif')


for i in range(50):
    #scelgo indice del paziente che voglio studiare
    paziente= i+1
    #scrivo il percorso
    if paziente<10:
        filename = home_path+"\sub-0"+str(paziente)+"\eeg\sub-0"+str(paziente)+"_task-rsvp_eeg.vhdr"
    if paziente >= 10:
        filename = home_path+"\sub-"+str(paziente)+"\eeg\sub-"+str(paziente)+"_task-rsvp_eeg.vhdr"
    #importo eeg
    raw=mne.io.read_raw_brainvision(filename, preload= True)
    #Applico un filtro FIR tra 0.1 e 12 Hz per isolare le frequenze rilevanti
    raw_filtred=raw.filter(0.1, 12, method='fir')
    #nuovo riferimento dei segnali differenzial degli elettrodi
    raw_new_ref=raw_filtred.set_eeg_reference(ref_channels='average')
    #Il dataset ha un sample ratio di 1000 Hz, che è troppo alto, quindi faccio un resampling e lo porto a 250 Hz
    raw_resampled=raw_new_ref.copy().resample(sfreq=250)
    #taglio la frequenza 10Hz 
    raw_final_notch=raw_resampled.copy().notch_filter(freqs=10)
    #salvo il segnale prodotto
    raw_final_notch.save(savename[i], overwrite=True)
    

Extracting parameters from C:\Users\Isabel\Desktop\TESI_git\sub-01\eeg\sub-01_task-rsvp_eeg.vhdr...


FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\Isabel\\Desktop\\TESI_git\\sub-01\\eeg\\sub-01_task-rsvp_eeg.vhdr'

In [ ]:
# Costruisco un ciclo for per memorizzare il segnale ricostruito dopo l'ICA di ciascun paziente
for i in range(1,51,1):
    notch = mne.io.read_raw_fif('paziente_' + str(i)+'_filtrato.fif') # recupero i file del filtro_notch eseguito sui pazienti
    
    preica = notch.copy().filter(l_freq=1., h_freq=None) # filtro passa-alto di 1 Hz
    
    bipolar_ref = mne.set_bipolar_reference(preica, 'Fp1', 'Fp2', ch_name='Reference', drop_refs=False) # creo un anale artificiale come pattern di riferimento per individuare l'ICs artefatti
    raw_ricostruito = bipolar_ref.copy() # copio il segnale contenente il nuovo canale creato
    
    ica = ICA(n_components=15, max_iter='auto', random_state=97)
    ica.fit(bipolar_ref) # genero l'unmixing matrix, con un numero di componenti indicato
    
    eog_indices, eog_scores = ica.find_bads_eog(bipolar_ref, ch_name='Reference') # Detect EOG related components using correlation.
    ica.exclude = eog_indices # escludo i canali che coincidono con il pattern EOG
    ica.apply(raw_ricostruito)
    
    # explained_var_ratio = ica.get_explained_variance_ratio(bipolar_ref)
    # explained_var_ratio
    
    final_raw = raw_ricostruito.drop_channels('Reference')
    final_raw.save('raw'+str(i)+'post-ica.fif')
    